In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical,normal

# env = gym.make("CartPole-v1")
env = gym.make("Hopper-v5")


# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [2]:
class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(11, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 3)  # 3 actions, mean for each action
        self.action_std = nn.Linear(128, 3)  # std for each action
        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.relu(self.action_head(x))
        action_std = F.softplus(self.action_std(x))
        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, action_std,state_values

In [3]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [4]:
gamma = 0.99 # discount factor for rewards
def select_action(state):
    state = torch.from_numpy(state).float()
    probs_mean, probs_std,state_value = model(state)
    actions=[]
    # create a categorical distribution over the list of probabilities of actions
    for i in range(len(probs_mean)):
        m = normal.Normal(probs_mean[i],probs_std[i])  # using normal distribution for sampling

    # and sample an action using the distribution
        action = m.sample()
        actions.append(action)
    action = torch.tensor(actions)
    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.numpy()


def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]



In [5]:

running_reward = 10
num_episodes = 10000
log_interval = 10
render=True
# run infinitely many episodes
for i_episode in range(num_episodes):

    # reset environment and episode reward
    state, _ = env.reset()
    ep_reward = 0

    # for each episode, only run 9999 steps so that we don't
    # infinite loop while learning
    for t in range(1, 10000):

        # select action from policy
        action = select_action(state)

        # take the action
        state, reward, terminated, truncated, _ = env.step(action)

        # if render:
        #     env.render()

        model.rewards.append(reward)
        ep_reward += reward
        if terminated or truncated:
            break

    # update cumulative reward
    running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

    # perform backprop
    finish_episode()

    # log results
    if i_episode % log_interval == 0:
        print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                i_episode, ep_reward, running_reward))

    # check if we have "solved" the cart pole problem
    if running_reward > env.spec.reward_threshold:
        print("Solved! Running reward is now {} and "
                "the last episode runs to {} time steps!".format(running_reward, t))
        break



Episode 0	Last reward: 37.15	Average reward: 11.36
Episode 10	Last reward: 5.03	Average reward: 14.99
Episode 20	Last reward: 9.13	Average reward: 15.87
Episode 30	Last reward: 81.96	Average reward: 21.34
Episode 40	Last reward: 13.72	Average reward: 20.71
Episode 50	Last reward: 10.95	Average reward: 20.94
Episode 60	Last reward: 10.90	Average reward: 20.96
Episode 70	Last reward: 28.47	Average reward: 22.07
Episode 80	Last reward: 6.31	Average reward: 18.01
Episode 90	Last reward: 10.13	Average reward: 18.48
Episode 100	Last reward: 12.18	Average reward: 15.09
Episode 110	Last reward: 3.05	Average reward: 18.23
Episode 120	Last reward: 10.56	Average reward: 17.41
Episode 130	Last reward: 3.30	Average reward: 17.25
Episode 140	Last reward: 14.67	Average reward: 16.39
Episode 150	Last reward: 1.18	Average reward: 15.39
Episode 160	Last reward: 3.76	Average reward: 20.23
Episode 170	Last reward: 8.73	Average reward: 20.37
Episode 180	Last reward: 45.49	Average reward: 20.25
Episode 190	